### Bibliotecas Utilizadas

In [27]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 

#Importar libs do programa
import design_functions
from v_p_cantoneira_flex.v_p_cantoneira_flex_solda import dim_cant_solda
from draw_autocad.draw_autocad_figures import *
from materials import * 

### Dimensionamento da ligação

#### Tipos de Aço escolhidos para o perfil

In [28]:
aço_escolhido = ASTM_A572

aço_escolhido_cantoneira = ASTM_A36

#### Propriedades geométricas do Perfil

In [29]:
perfil_escolhido = W_610x125_0
perfil_escolhido.inercias()
perfil_escolhido.material(aço_escolhido)

#### Propriedades  dos Parafusos

In [30]:
solda_escolhida = E70XX

#### Esforços solicitantes e solução

In [31]:
V = 572 #kN

T = 73 #kN


S = dim_cant_solda(T,V,cantoneiras_dict,aço_escolhido_cantoneira,perfil_escolhido,solda_escolhida,True,gamma)

In [32]:
cantoneira_escolhida = S[0]
espessura = S[1]

In [33]:
ver_chapa = cantoneira_escolhida.disp_vertices_chapa

### Criação do desenho

In [34]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing1.dwg


In [35]:
limpar_desenho(acad)

✅ Desenho limpo com sucesso.


#### Desenhar a cantoneira

In [36]:
objetos_s_cantoneira = desenhar_s_cantoneira(acad, cantoneira_escolhida, ver_chapa)

#### Desenhar seção das cantoneiras

In [37]:
# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 10, perfil_escolhido.t_w/2, (perfil_escolhido.h-cantoneira_escolhida.comprimento)/2  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

# Aplica a translação a todos os objetos na lista
for obj in objetos_s_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))

#### Desenhar seção do perfil

In [38]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_escolhido, posicao_x=-perfil_escolhido.b_f/2, posicao_y=-perfil_escolhido.h/2, altura_z=0)

# Rotacionar apenas a seção do perfil:
for obj in objetos_secao_perfil:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 0,0,perfil_escolhido.h/2  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

for obj in objetos_secao_perfil:
    obj.Move(APoint(0,0,0),vetor)

In [39]:
escrever_descricao(acad,0,0,perfil_escolhido.h + 10 ,"Cantoneira",cantoneira_escolhida.nome, perfil_escolhido.nome,espessura)